In [269]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import datetime

In [270]:
df_train=pd.read_csv('C:/Users/nguye/OneDrive/Desktop/DS103/data/raw/train.csv')
df_train.head()

C:\Users\nguye\AppData\Local\Temp\ipykernel_5460\3684304102.py:1: DtypeWarning: Columns (35,43) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train=pd.read_csv('C:/Users/nguye/OneDrive/Desktop/DS103/data/raw/train.csv')


,id,label,Field_1,Field_2,Field_3,Field_4,Field_5,Field_6,Field_7,Field_8,...,partner5_H,partner5_K,partner5_L,brief,num_of_phone,Field_78,Field_79,Field_80,Field_81,Field_82
0,0,1,2019-07-24T03:39:02.854Z,2019-07-31T20:10:02Z,1.0,GH,2018-12-27,2018-12-27,2019-01-01,2019-07-31,...,0.0,0.0,0.0,cb1,1.0,NaN,NaN,NaN,NaN,1
1,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,4,1.0,NaN,NaN,NaN,NaN,1
2,2,0,2019-01-17T07:15:26.367Z,2019-01-17T07:17:45Z,2.0,T1,2019-01-17,2019-01-17,2019-01-01,2019-12-31,...,0.0,0.0,0.0,1,1.0,33.0,10.769445,6.466667,0.0,2
3,3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1,1.0,33.0,10.769445,6.466667,0.0,3
4,4,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1,1.0,33.0,10.769445,6.466667,0.0,3


In [271]:
code_book=pd.DataFrame(df_train.columns,columns=['col_name'])

In [272]:
for i in range(len(code_book)):
    column_name = code_book['col_name'][i]
    domain_values = "{" + ",".join(str(x) for x in df_train[column_name].unique()) + "}"
    code_book.at[i, 'domain_values'] = domain_values
    code_book.at[i, 'N unique'] = len(df_train[column_name].unique())
    missing_values = df_train[column_name].isnull().sum()
    missing_percent = round((missing_values/len(df_train))*100,2)
    code_book.at[i, '%missing values'] = missing_percent   
code_book.to_excel('codebook_ver1.xlsx',index=False)

In [273]:
df_train['label'].value_counts()

0    36234
1    16796
Name: label, dtype: int64

In [274]:
import numpy as np
list_cv=df_train['maCv'].unique()
list_cv = [x for x in list_cv if x is not np.nan]
len(list_cv)

3589

In [275]:
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


# Biểu diễn văn bản dưới dạng vector số
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(list_cv)

# Tính cosine similarity
cos_sim = cosine_similarity(X)

# Phân cụm bằng KMeans
kmeans = KMeans(n_clusters=10)
kmeans.fit(cos_sim)

# Hiển thị các cụm
for i in range(10):
    cluster = np.array(list_cv)[kmeans.labels_ == i]
    print(f"Cluster {i}: {cluster}")


Cluster 0: ['Quản lý Nhà hàng' 'Bán hàng' 'Trưởng nhóm Bán hàng' 'NV bán hàng'
 'Tổ trưởng bán hàng' 'cửa hàng trưởng cửa hàng xăng dầu số 21'
 'giao hàng' 'Kế toán cửa hàng' 'Đại diện bán hàng' 'VM+ Cửa hàng trưởng'
 'Tổ trưởng Nhà hàng, Khối Dịch vụ ẩm thực' 'Giám sát bán hàng'
 'Tư vấn bán hàng xe DL KIA' 'NV Bán hàng'
 'NV Quan hệ khách hàng TDTT, làm việc tại Bắc Giang, Vùng 3'
 'Kế toán bán hàng' 'Giám sát nhà hàng'
 'Trưởng Nhóm Tiếp Nhận Yêu Cầu Khách Hàng' 'NV Bán hàng qua điện thoại'
 'CN kiểm hàng' 'Hàng rong' 'NV GIAO HàNG' 'Bán hàng vùng 2'
 'NV Phục Vụ Khách Hàng' '55741/NV Bán Hàng - CocaCola/Vùng 2'
 'Tài xế - giao hàng xe tải' 'Bán Hàng' 'Quản lÝ Cửa hàng'
 'NV Hỗ trợ Bán hàng' '39465/NV Bán hàng/Lenovo/vùng 02/Bạc Liêu'
 'Tổ Trưởng  hàng mẫu' 'Giám Sát Bán Hàng Trực Tiếp' 'CN Gấp Hàng'
 'CN Kiểm Hàng - Suối Tre - Long Khánh - Đồng Nai'
 'NV Chăm sóc khách hàng' 'Giám sát bán hàng (Nước)'
 '70629/NV Bán Hàng - CocaCola' 'Kỹ sư chất lượng khách hàng'
 'Nv bán hàng' 'NV 

In [276]:

def normalize_str(s):
    s = str(s).strip().lower()
    s = re.sub(' +', " ", s)
    return s
#phân nhóm nghề nghiệp
def categorize_job(string):
    if type(string) == str:
        blue_collar_matches = ['công nhân', 'cnv', 'cn', 'may công nghiệp', 'lao động', 'thợ', 'coõng nhaõn', 'c.n',
                               'lđ', 'bảo vệ', 'phụ kho', 'phục vụ', 'cán sự', 'vệ sỹ', 'phết keo', 'phụ việc', 'tạp vụ',
                               'gia công', 'cao su', 'kho', 'may', 'công', 'sửa', 'cong nhan', 'cảnh vệ', 'ép cao tần',
                               'đóng gói', 'giao nhận']
        teacher_matches = ['giáo viên', 'gv', 'gíao viên', 'giáo', 'giảng', 'bảo mẫu', 'cô nuôi']
        white_collar_matches = ['nhân viên', 'kế toán', 'cán bộ', 'nv', 'cb', 'nhõn viờn', 'giao dịch viên', 'operator',
                                'thủ', 'kinh doanh', 'văn thư', 'staff', 'trợ lý', 'bí thư', 'kinh tế', 'văn phòng',
                                'thư ký']
        driver_matches = ['tài xế', 'lái', 'tài xê', 'phụ xe']
        manager_matches = ['quản lý', 'phó phòng', 'hiệu phó', 'giám đốc', 'hiệu trưởng', 'chủ tịch', 'trưởng phòng']
        supervisor_matches = ['giám sát', 'tổ', 'cán sự']
        healthcare_matches = ['bác sĩ', 'dược', 'y sĩ', 'y sỹ', 'hộ sinh', 'y tá', 'dưỡng', 'hộ lý', 'bác sỹ', 'dịch tễ']
        chef_matches = ['bếp']
        engineer_matches = ['sư']
        specialist_matches = ['chuyên viên', 'chuyờn viờn', 'thanh tra', 'phiên dịch', 'kiểm ngân']
        technician_matches = ['kỹ thuật', 'kĩ thuật', 'ktv', 'kiểm soát', 'kỷ thuật', 'lập trình']
        entertainment_matches = ['diễn viên', 'ca sĩ', 'phim', 'chụp', 'entertainment', 'phóng viên']
        travel_matches = ['tiếp viên', 'hướng dẫn viên']
        fitness_matches = ['huấn luyện']
        customer_matches = ['bán', 'thu ngân', 'bán hàng', 'lễ tân', 'cửa hàng', 'tiếp thị', 'chăm sóc', 'tư vấn',
                            'cửa hàng']
        missing_matches = ['undefined', 'thieu chuc danh']

        if any(x in string for x in blue_collar_matches):
            return 'blue_collar'
        elif any(x in string for x in teacher_matches):
            return 'teacher'
        elif any(x in string for x in white_collar_matches):
            return 'white_collar'
        elif any(x in string for x in driver_matches):
            return 'driver'
        elif any(x in string for x in manager_matches):
            return 'manager'
        elif any(x in string for x in supervisor_matches):
            return 'supervisor'
        elif any(x in string for x in healthcare_matches):
            return 'healthcare_worker'
        elif any(x in string for x in chef_matches):
            return 'chef'
        elif any(x in string for x in engineer_matches):
            return 'engineer'
        elif any(x in string for x in specialist_matches):
            return 'specialist'
        elif any(x in string for x in technician_matches):
            return 'technician'
        elif any(x in string for x in entertainment_matches):
            return 'entertainment'
        elif any(x in string for x in travel_matches):
            return 'travel_worker'
        elif any(x in string for x in fitness_matches):
            return 'fitness'
        elif any(x in string for x in customer_matches):
            return 'customer_matches'
        elif any(x in string for x in missing_matches):
            return 'missing'
        else:
            return 'other'
    else:
        return string

df_train['job'] = df_train['maCv'].apply(normalize_str).apply(categorize_job).astype('category')
df_train = df_train.drop('maCv', axis=1)


In [277]:
#xử lý ngày tháng
def process_date_cols(data, cols):
    for col in cols:
        if col == 'Field_34':
            data[col + '_year'] = data[col].str[:4].astype('category')
            data[col + '_month'] = data[col].str[4:6].astype(float).astype('category')
        else:
            tmp = pd.to_datetime(data[col])
            data[col + '_year'] = tmp.dt.year.astype('category')
            data[col + '_month'] = tmp.dt.month.astype('category')
        data = data.drop(col, axis=1)
    return data

date_cols = ['Field_' + str(i) for i in [1, 2, 5, 6, 7, 8, 9, 11, 15, 25, 32, 33, 34, 35, 40, 43, 44]]
df_train=process_date_cols(df_train,date_cols)

In [278]:
"""có 2 cột giới tính, 2 cột này là chỉ giới tính cùng 1 người
    nếu cột này ko có, cột kia có tại index nào đó, thì cột nào có chính là giới tính thật của người đó
    nếu cả hai cột đều rỗng thì là missing data"""
def merge_gender_cols(data):
    if (data['gioiTinh'].lower() == data['info_social_sex']) & (data['gioiTinh'] != 'nan') & (data['info_social_sex'] != 'nan'):
        val = data['info_social_sex']
    elif (data['gioiTinh'] == 'nan') & (data['info_social_sex'] != 'nan'):
        val = data['info_social_sex']
    elif (data['gioiTinh'] != 'nan') & (data['info_social_sex'] == 'nan'):
        val = data['gioiTinh'].lower()
    elif data['gioiTinh'].lower() != data['info_social_sex']:
        val = 'other'
    else:
        val = 'missing'
    return val

def process_gender(data):
    data = (
        data
        .assign(gioiTinh=lambda x: x['gioiTinh'].astype(str),
                info_social_sex=lambda x: x['info_social_sex'].astype(str),
                gender=lambda x: x.apply(merge_gender_cols, axis=1))
    )
    return data

df_train=process_gender(df_train)
df_train=df_train.drop('gioiTinh',axis=1).drop('info_social_sex',axis=1)


In [279]:
#xác định tuổi của 1 người
def process_age(data):
    data['current_age'] = datetime.date.today().year - data['ngaySinh'].astype(str).str[:4].replace('nan', np.nan).astype(float)
    return data
df_train=process_age(df_train)
df_train=df_train.drop('ngaySinh',axis=1)

In [280]:
#có thể phân thành 2 nhóm là ko có thu nhập: 1, có thu nhập: 0
def categorize_welfare(string):
    if type(string) == str:
        matches = ['thất nghiệp', 'dịch vụ việc làm', 'bhtn', 'trợ cấp', 'nghèo', 'khó khăn', 'Bảo trợ xã hội',
                   'Ốm đau dài']
        if any(x in string for x in matches):
            return 1
        return 0

def process_welfare(data):
    data['Field_46_welfare'] = data['Field_46'].apply(normalize_str).apply(categorize_welfare)
    data['Field_61_welfare'] = data['Field_61'].apply(normalize_str).apply(categorize_welfare)
    data=data.drop('Field_46',axis=1).drop('Field_61',axis=1)
    return data

df_train=process_welfare(df_train)

In [281]:
#cỏ thể đưa về người có quốc tịch việt nam, nước ngoài, hoặc missing value
def categorize_location(x):
    if x != 'nan':
        if 'vietnam' in x:
            return 'VN'
        elif 'missing' in x:
            return 'missing'
        else:
            return 'International'


def process_country(data):
    data['homeTownCountry'] = data['homeTownCountry'].apply(normalize_str).apply(categorize_location).astype('category')
    data['currentLocationCountry'] = data['currentLocationCountry'].apply(normalize_str).apply(categorize_location).astype('category')
    return data

df_train=process_country(df_train)

In [282]:
#thay các missing values bằng "missing"

def replace_missing_level(data):
    for cols in data.columns:
        data[f"{cols}_new"] = data[cols].apply(lambda x: 'missing' if pd.isnull(x)  else x)
        data = data.drop(cols, axis=1)
    return data

df_train = replace_missing_level(df_train)



In [285]:
#xóa các dòng với một ngưỡng tỉ lệ missing values
def drop_missing_rows(data, thres):
    data = data.copy()
    data = (
        data
        .assign(row_missing_perc=lambda x: x.drop('label_new', axis=1).isnull().mean(axis=1))
        .query('row_missing_perc < {}'.format(thres))
        .drop('row_missing_perc', axis=1)
    )
    return data

df_train=drop_missing_rows(df_train,0.17)
df_train

,id_new,label_new,Field_3_new,Field_4_new,Field_10_new,Field_12_new,Field_13_new,Field_14_new,Field_16_new,Field_17_new,...,Field_40_year_new,Field_40_month_new,Field_43_year_new,Field_43_month_new,Field_44_year_new,Field_44_month_new,gender_new,current_age_new,Field_46_welfare_new,Field_61_welfare_new
0,0,1,1.0,GH,1.0,G8,1.0,1.0,1.0,1.0,...,2019.0,1.0,2017.0,1.0,2019.0,11.0,male,32.0,0,0
2,2,0,2.0,T1,1.0,missing,1.0,1.0,1.0,1.0,...,2019.0,1.0,2019.0,1.0,2019.0,8.0,female,25.0,0,0
6,6,0,2.0,T1,1.0,missing,1.0,1.0,1.0,1.0,...,2019.0,9.0,2017.0,1.0,2019.0,9.0,female,27.0,0,0
7,7,1,2.0,T1,1.0,G8,1.0,1.0,1.0,1.0,...,NaN,NaN,2017.0,2.0,2018.0,12.0,male,28.0,0,0
8,8,0,1.0,GH,1.0,missing,1.0,1.0,1.0,1.0,...,NaN,NaN,2017.0,3.0,2019.0,6.0,other,25.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53019,53019,0,missing,missing,missing,missing,missing,missing,missing,missing,...,NaN,NaN,NaN,NaN,NaN,NaN,female,missing,0,0
53021,53021,0,missing,missing,missing,missing,missing,missing,missing,missing,...,NaN,NaN,NaN,NaN,NaN,NaN,female,missing,0,0
53023,53023,0,2.0,T1,1.0,missing,1.0,1.0,1.0,1.0,...,2019.0,6.0,2017.0,3.0,2019.0,3.0,female,35.0,0,0
53025,53025,0,1.0,GH,1.0,missing,1.0,1.0,1.0,1.0,...,2020.0,1.0,2017.0,3.0,2019.0,8.0,female,35.0,0,0
